Install required libraries

In [ ]:
!pip install langchain
!pip install torch
!pip install sentence_transformers
!pip install faiss-cpu
!pip install CTransformers
!pip install huggingface-hub
!pip install pypdf
!pip -q install accelerate
!pip install llama-cpp-python
!pip -q install git+https://github.com/huggingface/transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.9 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=cbc001e5aa5e0ff8913b2df03287f43bd967ee1402

Import required libraries

In [ ]:
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import LlamaCpp, CTransformers
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain import PromptTemplate, LLMChain
import torch

Using the Mistral 7B model as backend

In [ ]:
model_url='https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q4_K_M.gguf'
model_name = "TheBloke/Mistral-7B-Instruct-v0.1-GGUF"
model_file = "mistral-7b-instruct-v0.1.Q4_K_M.gguf"

In [ ]:
#connect to google drive
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Using Langchain and integrating mistral LLM as backend

In [ ]:
config = {'max_new_tokens': 512, 'temperature': 0.1}

In [ ]:
llm = CTransformers(model=model_name, model_file=model_file, config=config)

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Demonstrate Simple Prompts

In [ ]:
sports_query = """Provide a brief description of the rules for BasketBall. Rules: {rules}"""
rules = "Basketball is a team sport in which two teams of five players each compete to score points by throwing a ball through the opposing team's hoop."
sports_prompt = PromptTemplate(template=sports_query, input_variables=["BasketBall", "rules"])
llm_chain_sports = LLMChain(prompt=sports_prompt, llm=llm)
response_sports = llm_chain_sports.run({"sport_name": "BasketBall", "rules": rules})

In [ ]:
response_sports

' The ball must be dribbled or passed, and cannot be carried more than two steps without being dribbled or passed. Players can try for a regular shot, worth two points, by shooting the ball into the hoop from above the rim. A successful free throw, worth one point, is awarded after a foul is committed. A three-point shot, worth three points, can be made by shooting the ball outside the three-point line. The team with the most points at the end of the game wins.'

In [ ]:
climate_query = """Provide a summary of recent developments in the field of Climate Change. Summary: {summary}"""
summary = "Recent developments in the field of climate change include international efforts to reduce carbon emissions, extreme weather events, and discussions on the impact of climate change on global ecosystems"
climate_prompt = PromptTemplate(template=climate_query, input_variables=["Climate Change", "summary"])
llm_chain_sports = LLMChain(prompt=climate_prompt, llm=llm)
response_climate = llm_chain_sports.run({"topic": "Climate Change", "summary": summary})

In [ ]:
response_climate

'. The Paris Agreement, signed by 195 countries in 2015, aims to limit global warming to well below 2 degrees Celsius above pre-industrial levels and to pursue efforts to limit the temperature increase to 1.5 degrees Celsius. Many countries have implemented policies to reduce their carbon emissions, such as transitioning to renewable energy sources and implementing carbon pricing mechanisms. However, extreme weather events, such as hurricanes, heatwaves, and droughts, continue to occur around the world, with some scientists linking these events to climate change. Additionally, discussions on the impact of climate change on global ecosystems, including the loss of biodiversity and changes in ocean acidification, are ongoing.'